In [1]:
# Descarga de datos a traves de la API de Kaggle

import os
import kaggle
import pandas as pd

# Configurar las credenciales de Kaggle
os.environ['KAGGLE_USERNAME'] = 'sergiogrigorow'
os.environ['KAGGLE_KEY'] = '53eaa416645f87b31ba66153190551e0'

# Autenticarse en Kaggle
kaggle.api.authenticate()

# Ruta donde se guardará el archivo descargado
directorio = 'C:/Users/sergi/Proyectos/Proyecto_Final/data/raw'
archivo_prueba = os.path.join(directorio, 'vehicles.csv')

if not os.path.exists(archivo_prueba):
    # Descargar y descomprimir el dataset
    kaggle.api.dataset_download_files('austinreese/craigslist-carstrucks-data', path=directorio, unzip=True)
else:
    pass



In [2]:
# Leer el archivo CSV descargado
directorio_csv = os.path.join(directorio, 'vehicles.csv')
df = pd.read_csv(directorio_csv,quotechar='"')

# Mostrar las primeras filas del DataFrame
print(df.head())

           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       NaN  ...  NaN   NaN

In [3]:
# Informacion del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [4]:
# Forma del dataframe
df.shape


(426880, 26)

In [5]:
# Se eliminan columnas con bajo valor predictivo o analitico
columnas_eliminar = ["id","county", "VIN", "region_url", "image_url", "description", "url", "cylinders","posting_date","title_status"]
df = df.drop(columns = columnas_eliminar)

In [6]:
# Forma del dataframe posterior a la eliminacion de columnas
df.shape

(426880, 16)

In [7]:
# Cantidad de valores nulos por columna
nulos = df.isnull().sum()
nulos


region               0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
fuel              3013
odometer          4400
transmission      2556
drive           130567
size            306361
type             92858
paint_color     130203
state                0
lat               6549
long              6549
dtype: int64

In [8]:
# Eliminar registros con valores nulos de columnas con pocos valores nulos
del_nulos = ['year','model','odometer','transmission','lat','long','manufacturer','fuel']
df = df.dropna(subset= del_nulos )
nulos = df.isnull().sum()
nulos

region               0
price                0
year                 0
manufacturer         0
model                0
condition       153385
fuel                 0
odometer             0
transmission         0
drive           117578
size            282111
type             82674
paint_color     118063
state                0
lat                  0
long                 0
dtype: int64

In [9]:
df.shape

(390219, 16)

In [10]:
# Completar datos faltantes en Type, a partir de la informacion del model, manufacturer, transmision y fuel
# Crear un diccionario para mapear las combinaciones de model, manufacturer, y transmission al type más frecuente
combination_to_type = df.groupby(['model', 'manufacturer', 'transmission','fuel'])['type'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

# Función para rellenar los valores nulos de type
def fill_type(row):
    if pd.isnull(row['type']):
        return combination_to_type.get((row['model'], row['manufacturer'], row['transmission'],row['fuel']), None)
    return row['type']

# Aplicar la función al DataFrame
df['type'] = df.apply(fill_type, axis=1)

nulos = df.isnull().sum()
nulos

region               0
price                0
year                 0
manufacturer         0
model                0
condition       153385
fuel                 0
odometer             0
transmission         0
drive           117578
size            282111
type              8664
paint_color     118063
state                0
lat                  0
long                 0
dtype: int64

In [11]:
# Eliminar los registros para los cuales no se pudo inferir un type en el paso anterior
df = df.dropna(subset=['type'])
nulos = df.isnull().sum()
nulos

region               0
price                0
year                 0
manufacturer         0
model                0
condition       147212
fuel                 0
odometer             0
transmission         0
drive           111824
size            273705
type                 0
paint_color     111431
state                0
lat                  0
long                 0
dtype: int64

In [12]:
# Completar datos faltantes en drive, a partir de la informacion del model, manufacturer, transmision, type y fuel
# Crear un diccionario para mapear las combinaciones de model, manufacturer, transmission, type y fuel al drive más frecuente

combination_to_drive = df.groupby(['model', 'manufacturer', 'transmission','type','fuel'])['drive'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

# Función para rellenar los valores nulos de drive
def fill_drive(row):
    if pd.isnull(row['drive']):
        return combination_to_drive.get((row['model'], row['manufacturer'], row['transmission'], row['type'],row['fuel']), None)
    return row['drive']

# Aplicar la función al DataFrame
df['drive'] = df.apply(fill_drive, axis=1)

nulos = df.isnull().sum()
nulos

region               0
price                0
year                 0
manufacturer         0
model                0
condition       147212
fuel                 0
odometer             0
transmission         0
drive            23630
size            273705
type                 0
paint_color     111431
state                0
lat                  0
long                 0
dtype: int64

In [13]:
# Eliminar los registros para los cuales no se pudo inferir un type en el paso anterior
df = df.dropna(subset=['drive'])
nulos = df.isnull().sum()
nulos

region               0
price                0
year                 0
manufacturer         0
model                0
condition       144073
fuel                 0
odometer             0
transmission         0
drive                0
size            250826
type                 0
paint_color     104416
state                0
lat                  0
long                 0
dtype: int64

In [14]:
# Se repite el mismo paso anterior, esta vez para encontrar los "size" faltantes
combination_to_size = df.groupby(['model', 'manufacturer'])['size'].apply(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()

def fill_size(row):
    if pd.isnull(row['size']):
        return combination_to_size.get((row['model'], row['manufacturer']), None)
    return row['size']

df['size'] = df.apply(fill_size, axis=1)

nulos = df.isnull().sum()
nulos

region               0
price                0
year                 0
manufacturer         0
model                0
condition       144073
fuel                 0
odometer             0
transmission         0
drive                0
size             51794
type                 0
paint_color     104416
state                0
lat                  0
long                 0
dtype: int64

In [15]:
df.shape

(357925, 16)

In [16]:
# Cantidad de valores unicos por cada columna
# Iterar sobre las columnas del DataFrame
for columna in df.columns:
    # Calcular la cantidad de valores únicos en cada columna
    cantidad_valores_unicos = df[columna].nunique()
    # Imprimir la cantidad de valores únicos en la columna actual
    print(f"Valores unicos en: '{columna}': {cantidad_valores_unicos}")


Valores unicos en: 'region': 404
Valores unicos en: 'price': 14594
Valores unicos en: 'year': 105
Valores unicos en: 'manufacturer': 41
Valores unicos en: 'model': 17377
Valores unicos en: 'condition': 6
Valores unicos en: 'fuel': 5
Valores unicos en: 'odometer': 96683
Valores unicos en: 'transmission': 3
Valores unicos en: 'drive': 3
Valores unicos en: 'size': 4
Valores unicos en: 'type': 13
Valores unicos en: 'paint_color': 12
Valores unicos en: 'state': 51
Valores unicos en: 'lat': 48515
Valores unicos en: 'long': 49020


In [ ]:
# Observamos gran diversidad de datos en la columna modelos.  
# Esto es algo que buscaremos solucionar en la siguiente etapa

In [17]:
df.to_csv("C:\\Users\\sergi\\Proyectos\\Proyecto_Final\\data\\processed\\vehicles_clean.csv", index=False)